In [13]:
# Import libraries
'''Main'''
import numpy as np
import pandas as pd
import os, time
import pickle, gzip

'''Data Viz'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl
%matplotlib inline

'''Data Prep and Model Evaluation'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

# Downloading the data
import tensorflow as tf
from keras.datasets import mnist

# Dimension Reductions
# Principal Component Analysis
from sklearn.decomposition import PCA
# Incremental PCA
from sklearn.decomposition import IncrementalPCA
# Sparse PCA
from sklearn.decomposition import SparsePCA
# Kernel PCA
from sklearn.decomposition import KernelPCA

# Singular Value Decomposition
from sklearn.decomposition import TruncatedSVD

# Gaussian Random Projection
from sklearn.random_projection import GaussianRandomProjection

# Sparse Random Projection
from sklearn.random_projection import SparseRandomProjection

# Isomap
from sklearn.manifold import Isomap

# Multidimensional Scaling
from sklearn.manifold import MDS

# Locally Linear Embedding (LLE)
from sklearn.manifold import LocallyLinearEmbedding

# t-SNE
from sklearn.manifold import TSNE

# Mini-batch dictionary learning
from sklearn.decomposition import MiniBatchDictionaryLearning

# Independent Component Analysis
from sklearn.decomposition import FastICA

import sys
# !{sys.executable} -m pip install fastcluster
# import fastcluster

import fastcluster
from scipy.cluster.hierarchy import dendrogram, cophenet
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import DBSCAN
# import hdbscan

In [11]:
# !{sys.executable} -m pip install hdbscan
# To upgrade numpy?
# !{sys.executable} pip install --upgrade numpy
import hdbscan

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached Cython-0.29.24-cp38-cp38-win_amd64.whl (1.7 MB)
Failed to build hdbscan


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\python.exe' 'C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\mtwpi\AppData\Local\Temp\tmp70yad95c'
       cwd: C:\Users\mtwpi\AppData\Local\Temp\pip-install-ntjgxedf\hdbscan_d436b5d4c6424733a79071af86bec4a1
  Complete output (27 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\hdbscan
  copying hdbscan\flat.py -> build\lib.win-amd64-3.8\hdbscan
  copying hdbscan\hdbscan_.py -> build\lib.win-amd64-3.8\hdbscan
  copying hdbscan\plots.py -> build\lib.win-amd64-3.8\hdbscan
  copying hdbscan\prediction.py -> build\lib.win-amd64-3.8\hdbscan
  copying hdbscan\robust_single_linkage_.py -> build\lib.win-amd64-3.8\hdbscan
  copying hdbscan\validity.py -> build\lib.win-amd64-3.8\hdbscan
  copying hdbsc

ModuleNotFoundError: No module named 'hdbscan'

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)


# Reshaping data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]**2))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1]**2))
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)

y_train = pd.Series(y_train)
y_test = pd.Series(y_test)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

# Put training data together
bigDF1 = pd.concat([X_train, y_train], axis = 1)

# Getting train and valid data
train, valid = train_test_split(bigDF1, test_size=0.2, random_state = 420, shuffle=True)

y_train = train.iloc[:, -1]
y_valid = valid.iloc[:, -1]

X_train = train.iloc[:, 0:-1]
X_valid = train.iloc[:, 0:-1]

# pd.concat([X_train, X_test])

(60000, 28, 28)
(60000, 784)
(60000,)
(10000,)


In [6]:
# Create Pandas DataFrames from the datasets
train_index = range(0,len(X_train))
validation_index = range(len(X_train), \
 len(X_train)+len(X_valid))
test_index = range(len(X_train)+len(X_valid), \
len(X_train)+len(X_valid)+len(X_test))

# pd.concat([X_train, y_train], axis = 1)

In [7]:
n_components = 784
whiten = False
random_state = 2018
pca = PCA(n_components=n_components, whiten=whiten, \
random_state=random_state)
X_train_PCA = pca.fit_transform(X_train)
X_train_PCA = pd.DataFrame(data=X_train_PCA, index=train_index)

We did not reduce dimensionality, but we'll choose the number of principal components.

In [8]:
# k-means - Inertia as the number of clusters varies
from sklearn.cluster import KMeans
n_clusters = 10
n_init = 10
max_iter = 300
tol = 0.0001
random_state = 420
n_jobs = 2
kMeans_inertia = pd.DataFrame(data=[],index=range(2,21), \
    columns=['inertia'])

for n_clusters in range(2,21):
    kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, \
        max_iter=max_iter, tol=tol, random_state=random_state, \
        n_jobs=n_jobs)
    cutoff = 99
    kmeans.fit(X_train_PCA.loc[:,0:cutoff])
    kMeans_inertia.loc[n_clusters] = kmeans.inertia_


C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"
C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"
C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"
C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be r

In [9]:
def analyzeCluster(clusterDF, labelsDF):
    countByCluster = \
        pd.DataFrame(data=clusterDF['cluster'].value_counts())
    
    # # rows by cluster
    countByCluster.reset_index(inplace=True,drop=False)
    countByCluster.columns = ['cluster','clusterCount']
    
    preds = pd.concat([labelsDF,clusterDF], axis=1)
    preds.columns = ['trueLabel','cluster']


    # observations by true label from train data
    countByLabel = pd.DataFrame(data=preds.groupby('trueLabel').count())
    
    countMostFreq = \
        pd.DataFrame(data=preds.groupby('cluster').agg( \
        lambda x:x.value_counts().iloc[0]))
    countMostFreq.reset_index(inplace=True,drop=False)
    countMostFreq.columns = ['cluster','countMostFrequent']
    
    accuracyDF = countMostFreq.merge(countByCluster, \
    left_on="cluster",right_on="cluster")
    overallAccuracy = accuracyDF.countMostFrequent.sum()/ \
    accuracyDF.clusterCount.sum()
    accuracyByLabel = accuracyDF.countMostFrequent/ \
        accuracyDF.clusterCount
    
    



In [7]:
n_clusters = 5
n_init = 10
max_iter = 300
tol = 0.0001
random_state = 2018
n_jobs = 2
kMeans_inertia = \
    pd.DataFrame(data=[],index=range(2,21),columns=['inertia'])
overallAccuracy_kMeansDF = \
    pd.DataFrame(data=[],index=range(2,21),columns=['overallAccuracy'])

for n_clusters in range(2,21):
    kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, \
        max_iter=max_iter, tol=tol, random_state=random_state, \
        n_jobs=n_jobs)
    cutoff = 99
    kmeans.fit(X_train_PCA.loc[:,0:cutoff])
    kMeans_inertia.loc[n_clusters] = kmeans.inertia_
    X_train_kmeansClustered = kmeans.predict(X_train_PCA.loc[:,0:cutoff])
    
    X_train_kmeansClustered = \
        pd.DataFrame(data=X_train_kmeansClustered, index=X_train.index, \
        columns=['cluster'])
    countByCluster_kMeans, countByLabel_kMeans, countMostFreq_kMeans, \
        accuracyDF_kMeans, overallAccuracy_kMeans, accuracyByLabel_kMeans \
        = analyzeCluster(X_train_kmeansClustered, y_train)
    overallAccuracy_kMeansDF.loc[n_clusters] = overallAccuracy_kMeans


C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


TypeError: cannot unpack non-iterable NoneType object

In [10]:
# k-means - Accuracy as the number of components varies
# On the original MNIST data (not PCA-reduced)
n_clusters = 20
n_init = 10
max_iter = 300
tol = 0.0001
random_state = 2018
n_jobs = 2
kMeans_inertia = pd.DataFrame(data=[],index=[9, 49, 99, 199, \
    299, 399, 499, 599, 699, 784],columns=['inertia'])
overallAccuracy_kMeansDF = pd.DataFrame(data=[],index=[9, 49, \
    99, 199, 299, 399, 499, 599, 699, 784], \
    columns=['overallAccuracy'])

for cutoffNumber in [9, 49, 99, 199, 299, 399, 499, 599, 699, 784]:
    kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, \
        max_iter=max_iter, tol=tol, random_state=random_state, \
        n_jobs=n_jobs)
    cutoff = cutoffNumber
    kmeans.fit(X_train.loc[:,0:cutoff])
    kMeans_inertia.loc[cutoff] = kmeans.inertia_
    X_train_kmeansClustered = kmeans.predict(X_train.loc[:,0:cutoff])
    X_train_kmeansClustered = pd.DataFrame(data=X_train_kmeansClustered, \
        index=X_train.index, columns=['cluster'])
    countByCluster_kMeans, countByLabel_kMeans, countMostFreq_kMeans, accuracyDF_kMeans, overallAccuracy_kMeans, accuracyByLabel_kMeans \
        = analyzeCluster(X_train_kmeansClustered, y_train)
    overallAccuracy_kMeansDF.loc[cutoff] = overallAccuracy_kMeans


C:\Users\mtwpi\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"
<ipython-input-10-3e618d752f7a>:20: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  kmeans.fit(X_train.loc[:,0:cutoff])


TypeError: cannot unpack non-iterable NoneType object

## Spawning A Dendogram (Upside-Down Tree)

In [ ]:
cutoff = 100
Z = fastcluster.linkage_vector(X_train_PCA.loc[:,0:cutoff], \
    method='ward', metric='euclidean')
Z_dataFrame = pd.DataFrame(data=Z, \
    columns=['clusterOne','clusterTwo','distance','newClusterSize'])


## Hierarchal Clustering

In [21]:
distance_threshold = 160
clusters = fcluster(Z, distance_threshold, criterion='distance')
X_train_hierClustered = \
    pd.DataFrame(data=clusters,index=X_train_PCA.index,columns=['cluster'])

NameError: name 'Z' is not defined

In [ ]:
eps = 3
min_samples = 5
leaf_size = 30
n_jobs = 4
db = DBSCAN(eps=eps, min_samples=min_samples, leaf_size=leaf_size,
    n_jobs=n_jobs)

cutoff = 99
X_train_PCA_dbscanClustered = db.fit_predict(X_train_PCA.loc[:,0:cutoff])
X_train_PCA_dbscanClustered = \
    pd.DataFrame(data=X_train_PCA_dbscanClustered, index=X_train.index, \
    columns=['cluster'])

countByCluster_dbscan, countByLabel_dbscan, countMostFreq_dbscan, \
    accuracyDF_dbscan, overallAccuracy_dbscan, accuracyByLabel_dbscan \
    = analyzeCluster(X_train_PCA_dbscanClustered, y_train)
overallAccuracy_dbscan

In [ ]:
min_cluster_size = 30
min_samples = None
alpha = 1.0
cluster_selection_method = 'eom'
hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, \
    min_samples=min_samples, alpha=alpha, \
    cluster_selection_method=cluster_selection_method)

cutoff = 10
X_train_PCA_hdbscanClustered = \
    hdb.fit_predict(X_train_PCA.loc[:,0:cutoff])

X_train_PCA_hdbscanClustered = \
    pd.DataFrame(data=X_train_PCA_hdbscanClustered, \
    index=X_train.index, columns=['cluster'])

countByCluster_hdbscan, countByLabel_hdbscan, \
    countMostFreq_hdbscan, accuracyDF_hdbscan, \
    overallAccuracy_hdbscan, accuracyByLabel_hdbscan \
    = analyzeCluster(X_train_PCA_hdbscanClustered, y_train)

